In [31]:
from elasticsearch import Elasticsearch, helpers
import requests

host = "http://localhost:9200"
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

# get documents
documents = []
for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)
 

es = Elasticsearch([host])
    
# Define the index settings and mappings
index_name = "my_index"
index_body = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"}
        }
    }
}
 
# Create the index with the defined settings and mappings
if not es.indices.exists(index=index_name):
    es.indices.create(index=index_name, body=index_body)
 
# Bulk index the documents
actions = [
    {
        "_index": index_name,
        "_source": doc
    }
    for doc in documents
]
 
helpers.bulk(es, actions)

(948, [])

In [32]:
# Define the search query
search_query = {
    "size": 1,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": "How do I execute a command in a running docker container?",
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            }
        }
    }
}
 
# Perform the search
response = es.search(index=index_name, body=search_query)
 
# Print the search results with scores
for hit in response['hits']['hits']:
    print(f"Score: {hit['_score']}, Document: {hit['_source']}")

Score: 84.050095, Document: {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}


In [33]:
# Define the search query
search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": "How do I execute a command in a running docker container?",  # Replace with the actual search query
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}
 
# Perform the search
response = es.search(index=index_name, body=search_query)
 
# Print the search results with scores
for hit in response['hits']['hits']:
    print(f"Score: {hit['_score']}, Document: {hit['_source']}")

Score: 84.050095, Document: {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}
Score: 51.04628, Document: {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani", 'section': '5. Deploying Machine Learning Models', 'question': 'How do I copy files from